# 🧹 Churn Prediction – Data Preprocessing Notebook
이 노트북은 `shop.db`에서 데이터를 불러와 고객 이탈 예측용 Feature Dataset을 생성합니다.


In [8]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import os
print('Imports 완료')

Imports 완료


## 1. DB 로드

In [15]:
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()
print('DB 연결 완료')

DB 연결 완료


In [16]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


[('customers',),
 ('sqlite_sequence',),
 ('products',),
 ('orders',),
 ('order_items',)]

In [26]:
import os
print("workdir:", os.getcwd())
print("files:", os.listdir())
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


workdir: c:\Users\dlwns\OneDrive\문서\GitHub\portfolio\Projects\project2_churn_prediction
files: ['.ipynb_checkpoints', 'churn_data_preprocessing.ipynb', 'churn_modeling.ipynb', 'data', 'images', 'README.md', 'shop.db']


[('customers',),
 ('sqlite_sequence',),
 ('products',),
 ('orders',),
 ('order_items',)]

## 2. 고객별 주문 집계

In [61]:
query = '''
SELECT
    customer_id,
    COUNT(order_id) AS total_orders,
    MIN(order_date) AS first_order,
    MAX(order_date) AS last_order
FROM orders
WHERE order_status = 'paid'
GROUP BY customer_id;
'''

df = pd.read_sql_query(query, conn)
print(df.shape)
df.head()


(1694, 4)


,customer_id,total_orders,first_order,last_order
0,1,3,2025-01-20 18:01:50,2025-05-28 01:38:26
1,2,1,2025-10-05 01:51:57,2025-10-05 01:51:57
2,3,1,2025-04-01 05:14:18,2025-04-01 05:14:18
3,4,1,2025-03-19 21:41:36,2025-03-19 21:41:36
4,5,1,2025-04-13 21:41:14,2025-04-13 21:41:14


## 3. Feature Engineering

In [69]:
df['first_order'] = pd.to_datetime(df['first_order'])
df['last_order'] = pd.to_datetime(df['last_order'])

# 첫 주문일 ~ 마지막 주문일까지 며칠 걸렸는지
df['days_between'] = (df['last_order'] - df['first_order']).dt.days

# 주문 간 평균 간격
df['avg_days_per_order'] = df['days_between'] / df['total_orders'].replace(0, 1)

# 이탈 라벨: 1회 주문 고객 = 1, 그 이상 = 0
df['churn_label'] = df['total_orders'].apply(lambda x: 1 if x == 1 else 0)

print(df.shape)
df.head()


(1694, 7)


,customer_id,total_orders,first_order,last_order,days_between,avg_days_per_order,churn_label
0,1,3,2025-01-20 18:01:50,2025-05-28 01:38:26,127,42.333333,0
1,2,1,2025-10-05 01:51:57,2025-10-05 01:51:57,0,0.000000,1
2,3,1,2025-04-01 05:14:18,2025-04-01 05:14:18,0,0.000000,1
3,4,1,2025-03-19 21:41:36,2025-03-19 21:41:36,0,0.000000,1
4,5,1,2025-04-13 21:41:14,2025-04-13 21:41:14,0,0.000000,1


## 4. Churn Label 생성 (1회 구매 고객 = 1)

In [70]:
df['churn_label'] = df['total_orders'].apply(lambda x: 1 if x==1 else 0)
df[['customer_id','total_orders','churn_label']].head()

,customer_id,total_orders,churn_label
0,1,3,0
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1


In [71]:
cursor.execute("SELECT COUNT(*) FROM orders;")
cursor.fetchall()


[(5000,)]

In [72]:
cursor.execute("SELECT order_status, COUNT(*) FROM orders GROUP BY order_status;")
cursor.fetchall()


[('paid', 3747), ('refund', 1253)]

In [73]:
cursor.execute("SELECT * FROM orders LIMIT 5;")
cursor.fetchall()


[(5001, 191, '2025-11-16 04:43:15', 'paid', 'bank', 'web'),
 (5002, 1828, '2025-08-21 06:14:35', 'paid', 'naver_pay', 'mobile'),
 (5003, 816, '2025-02-27 22:09:25', 'refund', 'bank', 'web'),
 (5004, 864, '2025-06-16 10:10:12', 'paid', 'kakao_pay', 'app'),
 (5005, 1238, '2025-11-14 23:21:28', 'paid', 'bank', 'mobile')]

## 5. CSV 저장

In [76]:
import os
os.makedirs('data', exist_ok=True)

df.to_csv('data/churn_dataset.csv', index=False)
print('data/churn_dataset.csv 저장 완료')


data/churn_dataset.csv 저장 완료


In [78]:
df = pd.read_csv('data/churn_dataset.csv')
print(df.shape)
print(df['churn_label'].value_counts())
df.head()


(1694, 7)
churn_label
0    1125
1     569
Name: count, dtype: int64


,customer_id,total_orders,first_order,last_order,days_between,avg_days_per_order,churn_label
0,1,3,2025-01-20 18:01:50,2025-05-28 01:38:26,127,42.333333,0
1,2,1,2025-10-05 01:51:57,2025-10-05 01:51:57,0,0.000000,1
2,3,1,2025-04-01 05:14:18,2025-04-01 05:14:18,0,0.000000,1
3,4,1,2025-03-19 21:41:36,2025-03-19 21:41:36,0,0.000000,1
4,5,1,2025-04-13 21:41:14,2025-04-13 21:41:14,0,0.000000,1
